# 2021/5/11 最適化 - 遺伝的アルゴリズム
###### OneMax問題を遺伝的アルゴリズムで解いてみる

In [22]:
import random
import copy

# パラメータ
gene_length = 20 # 遺伝子長
individual_length = 15 # 個体数
generation = 50 # 世代数
mutate_rate = 0.1 # 突然変異の確率
elite_rate = 0.2 # エリート選択の割合

### 下準備

In [23]:
def get_population():
    population = []
    for i in range(individual_length):
        population.append([random.randint(0,1) for j in range(gene_length)])
    return population

#15個体を作成
get_population()

[[0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1],
 [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1],
 [0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
 [0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1],
 [1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1],
 [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0]]

In [24]:
#合計値で評価
def fitness(pop):
    return sum(pop)

#評価結果と個体の組み合わせを作成する
[(fitness(p), p) for p in get_population()]

[(8, [0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]),
 (10, [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1]),
 (11, [1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0]),
 (10, [0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1]),
 (11, [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1]),
 (9, [1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1]),
 (9, [1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0]),
 (7, [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0]),
 (12, [1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1]),
 (10, [0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0]),
 (9, [1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]),
 (9, [0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0]),
 (8, [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1]),
 (13, [1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0]),
 (13, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1

In [25]:
#並べ替え
def evaluate(pop):
    pop.sort(reverse=True)
    return pop

evaluate([(fitness(p), p) for p in get_population()])

[(13, [1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1]),
 (13, [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0]),
 (13, [0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1]),
 (12, [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1]),
 (12, [1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1]),
 (12, [0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0]),
 (12, [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1]),
 (11, [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1]),
 (11, [1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1]),
 (9, [1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]),
 (9, [0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1]),
 (8, [1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]),
 (8, [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0]),
 (7, [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0]),
 (6, [0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 

In [26]:
#突然変異
def mutate(parent):
    r = random.randint(0, gene_length-1)
    child = copy.deepcopy(parent)
    child[r] = 1 if child[r]==0 else 0
    return child

#ランダムに一つの要素が反転する
pop=evaluate([(fitness(p), p) for p in get_population()])
print(pop[0][1])
print(mutate(pop[0][1]))

[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1]
[1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1]


In [27]:
#交配
#r1とr2間だけがparent2となる
def two_point_crossover(parent1, parent2):
    r1 = random.randint(0, gene_length-1)
    r2 = random.randint(r1, gene_length-1)
    child = copy.deepcopy(parent1)
    child[r1:r2] = parent2[r1:r2]
    return child

print(pop[0][1])
print(pop[1][1])
print(two_point_crossover(pop[0][1],pop[1][1]))

[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1]
[1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1]


### 実行

In [28]:
pop=evaluate([(fitness(p), p) for p in get_population()])

#現状を出力
print('Generation: 0')
print('Min : {}'.format(pop[-1][0]))
print('Max : {}'.format(pop[0][0]))
print('--------------------------')

for g in range(generation):
    print('Generation: ' + str(g+1))

    # エリートを選択
    eva = evaluate(pop) #並び替え
    elites = eva[:int(len(pop)*elite_rate)] #上位数個抽出
    
    print('')
    print('以下を抽出しました...')
    print(elites)
    
    pop=elites
    while len(pop) < individual_length:
        if random.random() < mutate_rate: #突然変異発生
            print('突然変異発生...')
            m=random.randint(0,len(elites)-1)
            child=mutate(elites[m][1])
            print((fitness(child),child))

        else:
            print('交配実施...')
            m1 = random.randint(0, len(elites)-1)
            m2 = random.randint(0, len(elites)-1)
            child = two_point_crossover(elites[m1][1], elites[m2][1])
            print((fitness(child),child))
            
        pop.append((fitness(child), child))   
        
    # 評価
    eva = evaluate(pop)
    pop = eva

    print('Min : {}'.format(pop[-1][0]))
    print('Max : {}'.format(pop[0][0]))
    print('--------------------------')
print('Result : {}'.format(pop[0]))   

Generation: 0
Min : 5
Max : 16
--------------------------
Generation: 1

以下を抽出しました...
[(16, [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1]), (12, [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1]), (12, [1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0])]
交配実施...
(16, [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1])
交配実施...
(12, [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1])
交配実施...
(14, [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1])
交配実施...
(13, [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1])
突然変異発生...
(11, [1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1])
交配実施...
(12, [1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0])
交配実施...
(14, [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1])
交配実施...
(12, [1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0])
交配実施...
(15, [1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1])
突然変異発生...
(12, [1, 0, 1, 1, 0, 1

(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
交配実施...
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
交配実施...
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
交配実施...
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
交配実施...
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
突然変異発生...
(19, [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
交配実施...
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
交配実施...
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
交配実施...
(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
突然変異発生...
(18, [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1])
突然変異発生...
(19, [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
Min : 18
Max : 20
--------------------------
Generation: 42

以下を抽出しました...
[(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), (20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1

# ここからはライブラリを読み込んで遺伝的アルゴリズムを実行する

In [36]:
#ライブラリ読み込み
import random
import numpy as np
from deap import base
from deap import creator
from deap import tools

In [37]:
#適応度クラスを作成
#FitnessMaxというクラス名、FitNessクラスを継承
#weight=1.0で大きいほど良い
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

#個体クラスの作成
#Individualはlistクラスを継承し、FitnessMaxを属性として持つ
creator.create("Individual", list, fitness=creator.FitnessMax)

C:\Users\nakam\anaconda3_1\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\nakam\anaconda3_1\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [38]:
#toolboxの作成
toolbox = base.Toolbox()

#遺伝子を作成する関数を登録
#初期個体の遺伝子を0,1で作成する
toolbox.register("attr_bool", random.randint, 0, 1)

#個体を生成する関数を登録
#initRepeatはattr_boolを20回実行する
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 20)

#個体集団を作成する関数を登録
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [39]:
def evalOneMax(individual):
    return sum(individual),

#評価関数を登録
toolbox.register("evaluate", evalOneMax)
#交叉を実行する関数を登録
toolbox.register("mate", tools.cxTwoPoints)
#変異を実行する関数を登録(Bitを反転させる)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
#次世代の個体を選択する関数を登録
toolbox.register("select", tools.selTournament, tournsize=3) #3個をランダムに選んで、一番良いものを残す

In [40]:
random.seed(64)

# 初期の個体群を生成
pop = toolbox.population(n=15)
print('初期状態は以下...')
print(np.array(pop))
print('')
print('評価結果は以下...')
print(np.array(pop).sum(axis=1))
CXPB, MUTPB, NGEN = 0.5, 0.2, 50 # 交差確率、突然変異確率、進化計算のループ回数

初期状態は以下...
[[1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 1 1 0 1 0 1 1 1 1 1 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 1 1 1]
 [1 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 1 0 1 1]
 [0 0 1 0 0 1 1 0 1 1 1 0 1 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0]
 [0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 1 0]
 [0 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 0]
 [0 0 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1]
 [1 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0]
 [0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1]
 [0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1]
 [0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0]
 [0 0 1 0 1 0 1 0 0 0 1 1 0 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0]]

評価結果は以下...
[ 6 10  7  9 10 11  8 12 13 12  9 15  5  8  8]


In [41]:
#初期の個体群の評価
fitnesses = list(map(toolbox.evaluate, pop)) #evaluate関数をpopに適用
print(fitnesses)

#ind(popの各サンプル)に評価結果を登録
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

[(6,), (10,), (7,), (9,), (10,), (11,), (8,), (12,), (13,), (12,), (9,), (15,), (5,), (8,), (8,)]


In [42]:
# 次世代の個体群を選択
offspring = toolbox.select(pop,len(pop))

# 個体群のクローンを生成（参照ではなくて、独立したコピーを作成）
offspring = list(map(toolbox.clone, offspring))

print('選択結果は以下...')
print(np.array(offspring))
print('')
print('評価結果は以下...')
print(np.array(offspring).sum(axis=1))

選択結果は以下...
[[0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0]
 [0 0 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1]
 [0 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 0]
 [0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1]
 [0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 1 0]
 [0 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 0]
 [0 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 0]
 [1 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0]
 [1 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0]
 [1 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 1 0 1 1]
 [0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1]
 [0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1]
 [0 0 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1]
 [0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0]
 [0 0 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1]]

評価結果は以下...
[11 13 12 15  8 12 12 12 12  9 15 15 13 11 13]


In [43]:
# 選択した個体群に交差と突然変異を適応する
# 偶数番目と奇数番目の個体を取り出して交差
for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if random.random() < CXPB:
        print('交配を実行します...')
        print('交配実行前は以下...')
        print(np.array(child1))
        print(np.array(child2))
        toolbox.mate(child1, child2)
        print('交配実行後は以下...')
        print(np.array(child1))
        print(np.array(child2))
        print('')
        
        #評価結果を削除
        del child1.fitness.values
        del child2.fitness.values

for mutant in offspring:
    if random.random() < MUTPB:
        print('変異を実行します...')
        print('変異実行前は以下...')
        print(mutant)
        toolbox.mutate(mutant)
        print('変異実行後は以下...')
        print(mutant)
        print('')
        
        #評価結果を削除
        del mutant.fitness.values
        
# 適合度（評価結果）が計算されていない個体を集めて適合度を計算
invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
fitnesses = map(toolbox.evaluate, invalid_ind)
for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit
    
print('進化結果は以下...')
print(np.array(offspring))
print('評価結果は以下...')
print(np.array(offspring).sum(axis=1))

交配を実行します...
交配実行前は以下...
[0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0]
[0 0 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1]
交配実行後は以下...
[0 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 0 0 0 0]
[0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1]

交配を実行します...
交配実行前は以下...
[0 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 0]
[1 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0]
交配実行後は以下...
[0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0]
[1 0 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 1 0 0]

変異を実行します...
変異実行前は以下...
[0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0]
変異実行後は以下...
[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0]

変異を実行します...
変異実行前は以下...
[0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0]
変異実行後は以下...
[0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0]

変異を実行します...
変異実行前は以下...
[0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1]
変異実行後は以下...
[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1]

変異を実行します...
変異実行前は以下...
[0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0]
変異実行後は以下...
[0, 1, 1, 1, 0, 

C:\Users\nakam\anaconda3_1\lib\site-packages\deap\tools\crossover.py:69: FutureWarning: tools.cxTwoPoints has been renamed. Use cxTwoPoint instead.
  FutureWarning)


# 上記を繰り返しながら、最適値を探す

In [265]:
import numpy as np
def main():
    random.seed(64)
    # 初期の個体群を生成
    pop = toolbox.population(n=15)
    print('初期状態')
    print(np.array(pop))
    CXPB, MUTPB, NGEN = 0.5, 0.2, 50 # 交差確率、突然変異確率、進化計算のループ回数

    print("Start of evolution")

    # 初期の個体群の評価
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # 進化計算開始
    for g in range(NGEN):
        print("-- Generation %i --" % g)

        # 次世代の個体群を選択
        offspring = toolbox.select(pop, len(pop))

        # 個体群のクローンを生成
        offspring = list(map(toolbox.clone, offspring))

        # 選択した個体群に交差と突然変異を適応する
        # 偶数番目と奇数番目の個体を取り出して交差
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # 適合度が計算されていない個体を集めて適合度を計算
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # 次世代群をoffspringにする
        pop[:] = offspring

        # すべての個体の適合度を配列にする
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    
if __name__ == "__main__":main()

初期状態
[[1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 1 1 0 1 0 1 1 1 1 1 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 1 1 1]
 [1 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 1 0 1 1]
 [0 0 1 0 0 1 1 0 1 1 1 0 1 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0]
 [0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 1 0]
 [0 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 0]
 [0 0 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1]
 [1 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0]
 [0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1]
 [0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1]
 [0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0]
 [0 0 1 0 1 0 1 0 0 0 1 1 0 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0]]
Start of evolution
  Evaluated 15 individuals
-- Generation 0 --
  Evaluated 7 individuals
  Min 8.0
  Max 15.0
  Avg 12.266666666666667
  Std 2.1123972690339783
-- Generation 1 --
  Evaluated 10 individuals
  Min 12.0
  Max 15.0
  Avg 13.4
  Std 1.0832051206181208
-- Generation 2 --
  Evaluated 13 individuals
  Min 12.0
  Max 16.0
  Avg 14.133333